In [1]:
pip install autograd

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from autograd import jacobian,hessian
import statsmodels.api as sm
import math
from random import choices

# 1(a)

In [3]:
#Read in daily price data on three stocks from the Internet
price_data = pd.read_csv('https://sites.google.com/site/chnyyang/price_data.csv')
price_data.head()

,Date,LMT,AMD,MSFT
0,2/11/2019,295.977448,22.959999,104.415909
1,2/12/2019,299.992310,22.820000,106.042915
2,2/13/2019,300.298126,22.850000,105.963547
3,2/14/2019,296.924438,23.129999,106.052834
4,2/15/2019,303.070068,23.680000,107.362373


In [4]:
# Find the period of daily price data: 2/11/2019-8/9/2019, 126 days in total
date = price_data.loc[:,'Date']
days = len(date)
#print(days)

#Read in the daily price data of LMT,AMD,MSFT seperately
LMT = price_data.loc[:,'LMT']
AMD = price_data.loc[:,'AMD']
MSFT = price_data.loc[:,'MSFT']

#Calculate the daily return of LMT,AMD,MSFT seperately
return_LMT = np.diff(LMT)/LMT[:-1]
return_AMD = np.diff(AMD)/AMD[:-1]
return_MSFT = np.diff(MSFT)/MSFT[:-1]

#Adding the daily return of LMT,AMD,MSFT to the original csv document
price_data['return_LMT'] = pd.DataFrame(return_LMT)
price_data['return_AMD'] = pd.DataFrame(return_AMD)
price_data['return_MSFT'] = pd.DataFrame(return_MSFT)

#You can look at the return of each stock by running the code in the next line
print(price_data)

          Date         LMT        AMD        MSFT  return_LMT  return_AMD  \
0    2/11/2019  295.977448  22.959999  104.415909    0.013565   -0.006098   
1    2/12/2019  299.992310  22.820000  106.042915    0.001019    0.001315   
2    2/13/2019  300.298126  22.850000  105.963547   -0.011234    0.012254   
3    2/14/2019  296.924438  23.129999  106.052834    0.020698    0.023779   
4    2/15/2019  303.070068  23.680000  107.362373   -0.006673    0.011402   
5    2/19/2019  301.047821  23.950001  107.312767    0.006619    0.000000   
6    2/20/2019  303.040497  23.950001  106.754837   -0.008756   -0.001253   
7    2/21/2019  300.386932  23.920000  109.006500    0.008604    0.018395   
8    2/22/2019  302.971436  24.360001  110.560745   -0.003777    0.014368   
9    2/25/2019  301.827148  24.709999  111.178452   -0.000294   -0.020235   
10   2/26/2019  301.738373  24.209999  111.945618    0.005558   -0.030153   
11   2/27/2019  303.415314  23.480000  111.756317    0.013197    0.002130   

# 1(b)

In [133]:
#Calculating the sample mean of each the three stocks
sample_mean_LMT = np.mean(return_LMT)
sample_mean_AMD = np.mean(return_AMD)
sample_mean_MSFT = np.mean(return_MSFT)
print("The sample mean of LMT is %.6f" %sample_mean_LMT)
print("The sample mean of AMD is %.6f" %sample_mean_AMD)
print("The sample mean of MSFT is %.6f" %sample_mean_MSFT)

#Calculating the covariance matrix
list = []
list.append(return_LMT)
list.append(return_AMD)
list.append(return_MSFT)
b = np.transpose(list)
covariance_matrix = np.cov(b, rowvar=False)
print("The covariance matrix is:") 
print(covariance_matrix)

The sample mean of LMT is 0.001997
The sample mean of AMD is 0.003748
The sample mean of MSFT is 0.002291
The covariance matrix is:
[[1.19719993e-04 8.22279244e-05 4.88370930e-05]
 [8.22279244e-05 1.15503385e-03 2.05980390e-04]
 [4.88370930e-05 2.05980390e-04 1.49416239e-04]]


# 1(c)

In [6]:
# Maximize expected return of the portfolio 
# r is the array of expected returns for each stock
# cov is the variance covariance matrix
# maxVar is the restriction on maximun variance of the portfolio
def maxReturn(r, cov, maxVar):
    #number of constraints
    n=3
    
    Q=np.zeros((len(r),len(r)));
    f=-r;
    c=0;

    H=[]
    k=[]
    d=[]

    for i in range(n):
        H.append(np.zeros((len(r),len(r))))
        k.append(np.zeros((len(r))))
        d.append(0)

    k[0]=np.array([1 for i in range(len(r))])
    d[0]=-1

    H[1]=2*cov
    d[1]=-maxVar


    #constraint setup
    ineq_cons={'type': 'ineq',
               'fun' : lambda p: np.array([-0.5*np.linalg.multi_dot([np.transpose(p),H[0],p])-np.dot(k[0],p)-d[0],
                                           -0.5*np.linalg.multi_dot([np.transpose(p),H[1],p])-np.dot(k[1],p)-d[1]])}
    #define objective function
    def objective(p):
        return 0.5*np.dot(np.transpose(p),np.dot(Q,p))+np.dot(f,p)+c
    jacb  = jacobian(objective)
    hessm = hessian(objective)

    #initial guesses
    g = len(r) #number of variables
    p0 = np.zeros(g) 


    # variable bounds assignment
    b = (0,1) #lower and upper bound
    bnds = tuple(b for i in range(len(r)))
    cons = ineq_cons #constraint assignment

    solution = minimize(objective,p0,method='trust-constr', jac=jacb ,hess=hessm,\
                         bounds=bnds,constraints=cons)
    x = solution.x


    # show optimal objective value
    print('Final Objective Value: %.7f' % -objective(x))

    # print solution
    print('Solution')
    for i in range(len(r)):
        print('p'+str(i+1)+' = %.7f' % x[i])
    
    return(-objective(x), x)


In [7]:
#Finding the maxium variance
stdv_LMT = np.std(return_LMT)
stdv_AMD = np.std(return_AMD)
stdv_MSFT = np.std(return_MSFT)
stdv_restriction = min(stdv_LMT,stdv_AMD,stdv_MSFT)
var_restriction = stdv_restriction**2

#Other inputs: exp_returns and variance matrix
exp_returns = np.array([sample_mean_LMT,sample_mean_AMD,sample_mean_MSFT])
var_cov_mtx = covariance_matrix
max_var = var_restriction

#Running the maximizing expected return framework
maxReturn(exp_returns, var_cov_mtx, max_var)

Final Objective Value: 0.0023369
Solution
p1 = 0.4229203
p2 = 0.1191918
p3 = 0.4564938


(0.002336881136123105, array([0.42292028, 0.11919179, 0.45649381]))

Conclusion: 0.42 unit should be invested in LMT, 0.12 unit should be invested in AMD and 0.46 unit should be invested in MSFT. The maximized daily return is 0.23%.

# 2(a)

In [8]:
#Read in dataset 'diamond.csv' from the Internet
diamond = pd.read_csv('https://sites.google.com/site/chnyyang/diamonds.csv')
diamond.head()

,color,carats,price
0,1,0.75,2050
1,1,1.00,4263
2,1,1.00,4272
3,1,1.01,4372
4,1,1.01,4372


In [192]:
# Multivariable linear regression
x1 = diamond.loc[:,'carats'] 
x2 = diamond.loc[:,'color'] 
y = diamond.loc[:,'price']

#Merge x1 and x2 into one variable and conduct linear regression
x1 = np.column_stack((x1,x2))  
x1 = sm.add_constant(x1)
model = sm.OLS(y,x1,missing='drop')

#Check the matrix of [1,x1,x2]
#print(x1)

# HC3 means we want heteroscedasticity robust standard errors
result = model.fit(cov_type='HC3')

print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.816
Model:                            OLS   Adj. R-squared:                  0.815
Method:                 Least Squares   F-statistic:                     843.2
Date:                Fri, 16 Aug 2019   Prob (F-statistic):          6.90e-140
Time:                        15:47:41   Log-Likelihood:                -2861.2
No. Observations:                 380   AIC:                             5728.
Df Residuals:                     377   BIC:                             5740.
Df Model:                           2                                         
Covariance Type:                  HC3                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const      -3196.0383    169.255    -18.883      0.0

In [194]:
# Reading standard errors  
se_ols = result.bse 
print(se_ols)
print("The robust error of the constant, carats and color is %.6f,%.6f,%.6f seperately." %(se_ols[0],se_ols[1],se_ols[2]))

const    169.254972
x1       149.937645
x2        53.013097
dtype: float64
The robust error of the constant, carats and color is 169.254972,149.937645,53.013097 seperately.


# 2(b)

In [11]:
carats = diamond.loc[:,'carats'] 
percentile25_carats = np.percentile(carats,25)
percentile50_carats = np.percentile(carats,50)
percentile75_carats = np.percentile(carats,75)
print("The 25 percent quantiles of carats is %.3f " % percentile25_carats)
print("The 50 percent quantiles of carats is %.3f " % percentile50_carats)
print("The 75 percent quantiles of carats is %.3f " % percentile75_carats)

The 25 percent quantiles of carats is 1.000 
The 50 percent quantiles of carats is 1.030 
The 75 percent quantiles of carats is 1.130 


# 2(c)

In [201]:
#Finding the diamonds whose "color" is equal to 1 and weight less than 25% quantile
sample_c1 = diamond.loc[(diamond['color']==1) & (diamond['carats'] <= percentile25_carats)]
#Choosing the price data of the chosen diamonds
price_c1 = sample_c1.loc[:,'price']
#price_c1.head()
variance_c1 = np.var(price_c1)

#Finding the diamonds whose "color" is equal to 1 and weight between 25% and 50% quantile
sample_c2 = diamond.loc[(diamond['color']==1) & (percentile25_carats < diamond['carats']) & (diamond['carats']<=percentile50_carats)]
#Choosing the price data of the chosen diamonds
price_c2 = sample_c2.loc[:,'price']
#price_c2.head()
variance_c2 = np.var(price_c2)

#Finding the diamonds whose "color" is equal to 1 and weight between 50% and 75% quantile
sample_c3 = diamond.loc[(diamond['color']==1) & (percentile50_carats < diamond['carats']) & (diamond['carats']<= percentile75_carats)]
#Choosing the price data of the chosen diamonds
price_c3 = sample_c3.loc[:,'price']
#price_c3.head()
variance_c3 = np.var(price_c3)

#Finding the diamonds whose "color" is equal to 1 and weight more than 75% quantile
sample_c4 = diamond.loc[(diamond['color']==1) & (percentile75_carats < diamond['carats'])]
#Choosing the price data of the chosen diamonds
price_c4 = sample_c4.loc[:,'price']
#price_c4.head()
variance_c4 = np.var(price_c4)

print("The conditional covariance of the price where the carat is less than 25 percent quantile is %.4f " %variance_c1)
print("The conditional covariance of the price where the carat is between 25 percent quantile and 50 percent quantile is %.4f " %variance_c2)
print("The conditional covariance of the price where the carat is between 50 percent quantile and 75 percenr quantile is %.4f " %variance_c3)
print("The conditional covariance of the price where the carat is more than 75 percent quantile is %.4f " %variance_c4)

The conditional covariance of the price where the carat is less than 25 percent quantile is 605255.5285 
The conditional covariance of the price where the carat is between 25 percent quantile and 50 percent quantile is 294669.1239 
The conditional covariance of the price where the carat is between 50 percent quantile and 75 percenr quantile is 201859.4439 
The conditional covariance of the price where the carat is more than 75 percent quantile is 490783.5544 


# 2(d)

In [198]:
#Reading the coefficients of the regression model
b_ols = result.params
#print(b_ols)
#The coefficient of "carats" is b_ols[1]
#The coefficient of "color" is b_ols[2]
#The coefficient of constant is b_ols[0]
β0 = b_ols[0]
β1 = b_ols[1]
β2 = b_ols[2]

#Calculating the variance of U among the diamonds whose "color" is equal to 1 and weight less than 25% quantile
x1_c1 = sample_c1.loc[:,'carats']
x2_c1 = sample_c1.loc[:,'color']
y_c1 = sample_c1.loc[:,'price']
uhat_c1 = y_c1 - (β0 + β1*x1_c1 + β2*x2_c1)
var_u_c1 = np.var(uhat_c1)
#print(var_u_c1)

#Calculating the variance of U among the diamonds whose "color" is equal to 1 and weight between 25% and 50% quantile
x1_c2 = sample_c2.loc[:,'carats']
x2_c2 = sample_c2.loc[:,'color']
y_c2 = sample_c2.loc[:,'price']
uhat_c2 = y_c2 - (β0 + β1*x1_c2 + β2*x2_c2)
var_u_c2 = np.var(uhat_c2)
#print(uhat_c2)

#Calculating the variance of U among the diamonds whose "color" is equal to 1 and weight between 50% and 75% quantile
x1_c3 = sample_c3.loc[:,'carats']
x2_c3 = sample_c3.loc[:,'color']
y_c3 = sample_c3.loc[:,'price']
uhat_c3 = y_c3 - (β0 + β1*x1_c3 + β2*x2_c3)
var_u_c3 = np.var(uhat_c3)
#print(uhat_c3)

#Calculating the variance of U among the diamonds whose "color" is equal to 1 and weight more than 75% quantile
x1_c4 = sample_c4.loc[:,'carats']
x2_c4 = sample_c4.loc[:,'color']
y_c4 = sample_c4.loc[:,'price']
uhat_c4 = y_c4 - (β0 + β1*x1_c4 + β2*x2_c4)
var_u_c4 = np.var(uhat_c4)
#print(uhat_c4)

print("The variance of U for diamonds whose color is equal to 1 and weight less than 25 percent quantile is %.5f" %var_u_c1)
print("The variance of U for diamonds whose color is equal to 1 and weight between 25 percent quantile and 50 percent quantile is %.5f" %var_u_c2)
print("The variance of U for diamonds whose color is equal to 1 and weight between 50 percent quantile and 75 percent quantile is %.5f" %var_u_c3)
print("The variance of U for diamonds whose color is equal to 1 and weight more than 75 percent quantile is %.5f" %var_u_c4)
print("Conclusion: The variance of U increases when carat increases.")

The variance of U for diamonds whose color is equal to 1 and weight less than 25 percent quantile is 74539.98924
The variance of U for diamonds whose color is equal to 1 and weight between 25 percent quantile and 50 percent quantile is 239126.40605
The variance of U for diamonds whose color is equal to 1 and weight between 50 percent quantile and 75 percent quantile is 312353.25616
The variance of U for diamonds whose color is equal to 1 and weight more than 75 percent quantile is 375155.89054
Conclusion: The variance of U increases when carat increases.


# 2(e)

In [199]:
# Mean of price for diamonds with weight 50% quantile
sample_e = diamond.loc[(diamond['carats']==percentile50_carats)]
color_e = sample_e.loc[:,'color']
mean_price = β0+β1*percentile50_carats+β2*np.mean(color_e)

print("The mean price at 50 percent quantile of the weight is %.4f" %mean_price)

The mean price at 50 percent quantile of the weight is 4059.8288


# 2(f)

In [200]:
#Confidence interval for specific predictions
color = diamond.loc[:,'color'] 
#Estimation of price(y)
yhat = β0 + β1*carats + β2*color
#two variables in the regrssion model, so ddof=2
#Standard error of prediction
se_yhat = np.std(yhat)/len(color)

#Standard error of regression
price_e = sample_e.loc[:,'price']
color_e = sample_e.loc[:,'color']
uhat = price_e - (β0 + β1*percentile50_carats + β2*color_e)
ser = np.var(uhat)

#For carats = percentile50_carats(assuming confidence interval is 95%) 
CI_upper = mean_price +1.96*math.sqrt(se_yhat**2+ser)
CI_lower = mean_price -1.96*math.sqrt(se_yhat**2+ser)

print("The confidence interval is (%.6f,%.6f) " %(CI_lower,CI_upper))

The confidence interval is (2579.574931,5540.082649) 


# 2(g)

As the variance of U increases when carat increases, so it is not reasonable to assume homoskedasticity.

# 2(h)

As it is not reasonable to assume homoskedasticity in this regression model, it is not reasonable to calculate Var(U) and use it calculate the confidence interval of the regression model for some values of the explanatory variable.

# 3(a)

In [126]:
#Read in the data of the market returns and the adjusted closing price of HD, AAPL and VZ) from the Internet
capm = pd.read_csv('https://sites.google.com/site/chnyyang/capm.csv')
capm.head()

,Date,Mkt-Rf,Rf,HD,AAPL,VZ
0,1/2/1990,-0.0785,0.0057,1.31,7.83,8.41
1,2/1/1990,0.0111,0.0057,1.49,7.85,7.73
2,3/1/1990,0.0183,0.0064,1.71,9.29,8.12
3,4/2/1990,-0.0336,0.0069,1.75,9.09,8.15
4,5/1/1990,0.0842,0.0068,2.27,9.55,9.08


In [127]:
#Reading the adjusted closing price of HD,AAPL and VZ
HD = capm.loc[:,'HD']
AAPL = capm.loc[:,'AAPL']
VZ = capm.loc[:,'VZ']

#Calculate the monthly return of LMT,AMD,MSFT seperately
return_HD = np.diff(HD)/HD[:-1]
return_AAPL = np.diff(AAPL)/AAPL[:-1]
return_VZ = np.diff(VZ)/VZ[:-1]

#Adding the monthly return of LMT,AMD,MSFT to the original csv document
capm['return_HD'] = pd.DataFrame(return_HD)
capm['return_AAPL'] = pd.DataFrame(return_AAPL)
capm['return_VZ'] = pd.DataFrame(return_VZ)

#You can look at the return of each stock by running the code in the next line
print(capm)

          Date  Mkt-Rf      Rf     HD   AAPL     VZ  return_HD  return_AAPL  \
0     1/2/1990 -0.0785  0.0057   1.31   7.83   8.41   0.137405     0.002554   
1     2/1/1990  0.0111  0.0057   1.49   7.85   7.73   0.147651     0.183439   
2     3/1/1990  0.0183  0.0064   1.71   9.29   8.12   0.023392    -0.021529   
3     4/2/1990 -0.0336  0.0069   1.75   9.09   8.15   0.297143     0.050605   
4     5/1/1990  0.0842  0.0068   2.27   9.55   9.08  -0.048458     0.084817   
5     6/1/1990 -0.0110  0.0063   2.16  10.36   8.65  -0.111111    -0.060811   
6     7/2/1990 -0.0190  0.0068   1.92   9.73   8.71  -0.010417    -0.117163   
7     8/1/1990 -0.1015  0.0066   1.90   8.59   7.54  -0.100000    -0.215367   
8     9/4/1990 -0.0612  0.0060   1.71   6.74   8.33  -0.035088     0.059347   
9    10/1/1990 -0.0192  0.0068   1.65   7.14   9.32   0.218182     0.198880   
10   11/1/1990  0.0634  0.0057   2.01   8.56   9.70   0.079602     0.170561   
11   12/3/1990  0.0246  0.0060   2.17  10.02   9.57 

# 3(b)

In [128]:
#Calculating Rmkt and add it to "capm.csv"
Rmkt = capm.loc[:,'Mkt-Rf'] + capm.loc[:,'Rf']
capm['Rmkt'] = pd.DataFrame(Rmkt)

#Calculating β(HD)
#Choosing the non-Nan value in monthly return of HD, for our calculation, just omit the last one in variable "Rmkt", as daily return missed one value 
return_HD = return_HD[~np.isnan(return_HD)]
cov_HD = np.cov(return_HD,Rmkt[0:len(Rmkt)-1])
cov_HD = cov_HD[0,1]
β_HD = cov_HD/np.var(Rmkt)
#print(β_HD)  

#Calculate β(AAPL)
#Choosing the non-Nan value in monthly return of AAPL, for our calculation, just omit the last one in variable "Rmkt", as daily return missed one value 
return_AAPL = return_AAPL[~np.isnan(return_AAPL)]
cov_AAPL = np.cov(return_AAPL,Rmkt[0:len(Rmkt)-1])
cov_AAPL = cov_AAPL[0,1]
β_AAPL = cov_AAPL/np.var(Rmkt)
#print(β_AAPL) 

#Calculate β(VZ)
#Choosing the non-Nan value in monthly return of VZ, for our calculation, just omit the last one in variable "Rmkt", as daily return missed one value 
return_VZ = return_VZ[~np.isnan(return_VZ)]
cov_VZ = np.cov(return_VZ,Rmkt[0:len(Rmkt)-1])
cov_VZ = cov_VZ[0,1]
β_VZ = cov_VZ/np.var(Rmkt)
#print(β_VZ)

print("The β of HD is %.5f " % β_HD)
print("The β of AAPL is %.5f " % β_AAPL)
print("The β of VZ is %.5f " % β_VZ)

The β of HD is -0.05836 
The β of AAPL is -0.04861 
The β of VZ is -0.17695 


# 3(c)

The β of HD is -0.05836 means that when the market index decreases by 1%, the price of HD increases 0.058%; 
The β of AAPL is -0.04861 means that when the market index decreases by 1%, the price of HD increases 0.049%; 
The β of VZ is -0.17695 means that when the market index decreases by 1%, the price of HD increases 0.177%.

# 3(d)

In [163]:
np.random.seed(120)
def bootSE(sample):
    sample = np.array(sample)
    sample = sample[~np.isnan(sample)]
    
    M = 200
    
    # Bootstrap
    var_boot = np.array([])
    data=[]
    for bn in range(M):
        data_bootn = choices(sample,k=len(sample))
        data.append(np.array([data_bootn]))
        
    # the function only return the bootstrapped dataset
    return(data)

# Input into the bootstrap function called "bootSE" , for HD only 
#Step 1: Creating the tuple(Rmkt,RHD)
sample_data_HD = pd.DataFrame({'Rmkt':Rmkt,'RHD':return_HD})
sample_data_list_HD = sample_data_HD.values.tolist()
sample_data_tuple_HD = tuple(sample_data_list_HD)
#print(sample_data_tuple_HD)

#Step 2: Getting 2000 “reshuffled” data for stock HD
bootstrapped_data_HD = bootSE(sample_data_tuple_HD)
#print(bootstrapped_data_HD)

# Input into the bootstrap function called "bootSE" , for AAPL only 
#Step 1: Creating the tuple(Rmkt,RAAPL)
sample_data_AAPL = pd.DataFrame({'Rmkt':Rmkt,'RAAPL':return_AAPL})
sample_data_list_AAPL = sample_data_AAPL.values.tolist()
sample_data_tuple_AAPL = tuple(sample_data_list_AAPL)
#print(sample_data_tuple_AAPL)

#Step 2: Getting 2000 “reshuffled” data for stock AAPL
bootstrapped_data_AAPL = bootSE(sample_data_tuple_AAPL)
#print(bootstrapped_data_AAPL)

# Input into the bootstrap function called "bootSE" , for VZ only 
#Step 1: Creating the tuple(Rmkt,RVZ)
sample_data_VZ = pd.DataFrame({'Rmkt':Rmkt,'RVZ':return_VZ})
sample_data_list_VZ = sample_data_VZ.values.tolist()
sample_data_tuple_VZ = tuple(sample_data_list_VZ)
#print(sample_data_tuple_VZ)

#Step 2: Getting 2000 “reshuffled” data for stock VZ
bootstrapped_data_VZ = bootSE(sample_data_tuple_VZ)
#print(bootstrapped_data_VZ)

# 3(e)

In [180]:
#The framework of bootstrap
np.random.seed(120)
def bootSE(sample):
    sample = np.array(sample)
    #Only the last line of each tuple is NaN
    sample = sample[0:len(sample)-1]
    #Choosing the covariance in the covariance-variance matrix and calculate β, β is "calculated_statistic" in this command
    calculated_statistic = np.cov(sample,rowvar=False)[0,1]/np.var(Rmkt)   
    
    M = 2000
    
    # Bootstrap
    var_boot = np.array([])
    data = []
    for bn in range(M):
        data_bootn = choices(sample,k=len(sample))
        data.append(np.array([data_bootn]))
        var_boot=np.append(var_boot,np.cov(data_bootn,rowvar=False)[0,1]/np.var(Rmkt))
    
    average_of_variances = np.average(var_boot)
    
    se = np.sqrt(np.sum((var_boot - average_of_variances)**2)/(M-1))
    
    # the function returns two results: the statistic and the standard error
    return(calculated_statistic,se,var_boot)

#2000 estimates of the bootstrapped estimates of βHD
calculated_statistic,se,var_boot = bootSE(sample_data_tuple_HD) 
var_boot_HD = var_boot
se_HD = se
print("The bootstrapped estimate of βHD is: ")
print(var_boot_HD)
#print(len(var_boot_HD))    #Checking how many elements are in the bootstrapped estimate

#2000 estimates of the bootstrapped estimates of βAAPL
calculated_statistic,se,var_boot = bootSE(sample_data_tuple_AAPL) 
var_boot_AAPL = var_boot
se_AAPL = se
print("The bootstrapped estimate of βAAPL is: ")
print(var_boot_AAPL)
#print(len(var_boot_AAPL))    #Checking how many elements are in the bootstrapped estimate

#2000 estimates of the bootstrapped estimates of βVZ
calculated_statistic,se,var_boot = bootSE(sample_data_tuple_VZ) 
var_boot_VZ = var_boot
se_VZ = se
print("The bootstrapped estimate of βVZ is: ")
print(var_boot_VZ)
#print(len(var_boot_VZ))    #Checking how many elements are in the bootstrapped estimate

The bootstrapped estimate of βHD is: 
[ 0.2776048  -0.12549674  0.35189212 ...  0.07418326 -0.28899981
 -0.03729608]
The bootstrapped estimate of βAAPL is: 
[-0.19687588  0.31495252 -0.63176099 ... -0.26058417  0.45489346
 -0.49606759]
The bootstrapped estimate of βVZ is: 
[-0.22870413 -0.1467473  -0.2244584  ... -0.39847692  0.00289365
 -0.12557403]


# 3(f)

In [184]:
#Computing the 2.5% quantile and 97.5% quantile for βHD
CI_HD_lower = np.percentile(var_boot_HD,0.25)
CI_HD_upper = np.percentile(var_boot_HD,97.5)
print("the 0.025 quantiles for βHD is %.6f and 0.975 quantiles for βHD is %.6f " %(CI_HD_lower,CI_HD_upper))
print("So the bootstrapped Confidence Interval of βHD is (%.6f,%.6f) " %(CI_HD_lower,CI_HD_upper))

#Computing the 2.5% quantile and 97.5% quantile for βAAPL
CI_AAPL_lower = np.percentile(var_boot_AAPL,0.25)
CI_AAPL_upper = np.percentile(var_boot_AAPL,97.5)
print("the 0.025 quantiles for βAAPL is %.6f and 0.975 quantiles for βAAPL is %.6f " %(CI_AAPL_lower,CI_AAPL_upper))  
print("So the bootstrapped Confidence Interval of βAAPL is (%.6f,%.6f) " %(CI_AAPL_lower,CI_AAPL_upper))

#Computing the 2.5% quantile and 97.5% quantile for βVZ
CI_VZ_lower = np.percentile(var_boot_VZ,0.25)
CI_VZ_upper = np.percentile(var_boot_VZ,97.5)
print("the 0.025 quantiles for βVZ is %.6f and 0.975 quantiles for βVZ is %.6f " %(CI_VZ_lower,CI_VZ_upper))  
print("So the bootstrapped Confidence Interval of βVZ is (%.6f,%.6f) " %(CI_VZ_lower,CI_VZ_upper))

the 0.025 quantiles for βHD is -0.547527 and 0.975 quantiles for βHD is 0.305929 
So the bootstrapped Confidence Interval of βHD is (-0.547527,0.305929) 
the 0.025 quantiles for βAAPL is -1.027986 and 0.975 quantiles for βAAPL is 0.599707 
So the bootstrapped Confidence Interval of βAAPL is (-1.027986,0.599707) 
the 0.025 quantiles for βVZ is -0.665702 and 0.975 quantiles for βVZ is 0.139120 
So the bootstrapped Confidence Interval of βVZ is (-0.665702,0.139120) 


# 3(g)

For HD and β of AAPL, we can not reject the null hypothesis "β=0", as the estimated β is in the bootstrapped confidence intetval; while for VZ, we can reject the null hypothesis "β=0", as the estimated β is out of the bootstrapped confidence intetval.

# 3(h)

In [204]:
# Univariable linear regression
y = capm.loc[:,'return_HD'] 
x = capm.loc[:,'Rmkt']

x = sm.add_constant(x)
model = sm.OLS(y,x,missing='drop')
#print(x)

# HC3 means we want heteroscedasticity robust standard errors
result2 = model.fit(cov_type='HC3')

print(result2.summary())

                            OLS Regression Results                            
Dep. Variable:              return_HD   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.008
Method:                 Least Squares   F-statistic:                   0.08903
Date:                Fri, 16 Aug 2019   Prob (F-statistic):              0.766
Time:                        19:18:14   Log-Likelihood:                 131.79
No. Observations:                 119   AIC:                            -259.6
Df Residuals:                     117   BIC:                            -254.0
Df Model:                           1                                         
Covariance Type:                  HC3                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0356      0.008      4.466      0.0

# 3(i)

The estimates should be equal to the β calculated in quesion(b), as CAPM can be written as E(Ri)=βi*E(Rmkt)+(1-βi)*Rf. In this form, βi = βHD in question(h).

# 3(j)

In [244]:
#Verifing whether OLS can be used in this regrssion model is verifing whether the regression model has homoskedasticity or not
#Reading the results of the regression in question 3(h)
b_ols = result2.params  # actual estimates
α = b_ols[0]
β = b_ols[1]
y = capm.loc[:,'return_HD'] 
x = capm.loc[:,'Rmkt']


#Calculating uhat and its variance
uhat = y - (α + β*x)
uhat_x = pd.DataFrame({'uhat':uhat,'return_HD':x})
uhat_list_x = uhat_x.values.tolist()
uhat_tuple_x = tuple(uhat_list_x)
uhat_array_x = np.array(uhat_tuple_x)
covmtx_x_uhat = np.cov(uhat_array_x[0:len(uhat_array_x)-1],rowvar=False)
print("The covariance between uhat and x is %.6f" %covmtx_x_uhat[0,1])
print("Conclusion: Equation (2) can be consistently estimated with OLS")


The covariance between uhat and x is 0.000000
Conclusion: Equation (2) can be consistently estimated with OLS
